In [84]:
import pandas as pd

In [85]:
def get_elements_by_pattern(elements: list[str], beginning, end) -> list[str]:
    orig_query_line = list(map(
        lambda s: s[s.index(beginning) + len(beginning) : s.index(end, s.index(beginning))],
        filter(
            lambda s: beginning in s and end in s and s.index(beginning) < s.index(end, s.index(beginning)),
            elements
        )
    ))
    
    # make sure exactly one output of this type can be found
    assert len(orig_query_line) > 0, f"Found {len(orig_query_line)} queries but expected at least 1!"
    return orig_query_line

def get_number_of_occurences_by_pattern(elements: list[str], pattern: str) -> list[int]:
    return list(map(lambda x: x.count(pattern), elements))

    

In [ ]:
import subprocess

cmd = """
cd ../..

export RUST_LOG=warn
cargo build
for i in {1..2}; do
    cargo run --package reducer --bin reducer -- --query queries/query$i/original_test.sql --test src/resources/native.sh
done
"""

result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
output = result.stdout + result.stderr  # Capture both stdout and stderr if needed

In [79]:
analysis = pd.DataFrame()
output

'Current directory: /workspaces/reducer\n\x1b\x1b\x1bwarning\x1b\x1b\x1b: unused import: `std::sync::OnceLock`\x1b\n\x1b \x1b\x1b\x1b\x1b--> \x1b\x1bsrc/driver.rs:8:5\x1b\n\x1b  \x1b\x1b\x1b\x1b|\x1b\n\x1b\x1b\x1b8\x1b\x1b \x1b\x1b\x1b\x1b|\x1b\x1b \x1b\x1buse std::sync::OnceLock;\x1b\n\x1b  \x1b\x1b\x1b\x1b|\x1b\x1b     \x1b\x1b\x1b\x1b^^^^^^^^^^^^^^^^^^^\x1b\n\x1b  \x1b\x1b\x1b\x1b|\x1b\n\x1b  \x1b\x1b\x1b\x1b= \x1b\x1b\x1bnote\x1b\x1b: `#[warn(unused_imports)]` on by default\x1b\n\n\x1b\x1b\x1bwarning\x1b\x1b\x1b: unused import: `log::info`\x1b\n\x1b \x1b\x1b\x1b\x1b--> \x1b\x1bsrc/parser.rs:1:5\x1b\n\x1b  \x1b\x1b\x1b\x1b|\x1b\n\x1b\x1b\x1b1\x1b\x1b \x1b\x1b\x1b\x1b|\x1b\x1b \x1b\x1buse log::info;\x1b\n\x1b  \x1b\x1b\x1b\x1b|\x1b\x1b     \x1b\x1b\x1b\x1b^^^^^^^^^\x1b\n\n\x1b\x1b\x1bwarning\x1b\x1b\x1b: unused import: `crate::driver`\x1b\n\x1b \x1b\x1b\x1b\x1b--> \x1b\x1bsrc/reducer.rs:2:5\x1b\n\x1b  \x1b\x1b\x1b\x1b|\x1b\n\x1b\x1b\x1b2\x1b\x1b \x1b\x1b\x1b\x1b|\x1b\x1b \x1b\x1buse 

In [80]:
results = list(filter(lambda x: "[ANALYSIS]" in x, output.split("\n")))
results

['[2025-05-27T11:42:23Z WARN  reducer] [ANALYSIS] QUERY PATH: "/workspaces/reducer/queries/query2/original_test.sql"[END ANALYSIS]',
 '[2025-05-27T11:42:23Z WARN  reducer] [ANALYSIS] ORIGINAL QUERY: "CREATE TABLE IF NOT EXISTS t_DX44 (c_LGUf NUMERIC, c_Hlmf3w REAL DEFAULT 749171.692897985, c_ewZ TEXT, c_EwP TEXT DEFAULT \'Fn58MvfLqzQ2DMC4\', c_YBA7sBV TEXT CHECK (length(c_YBA7sBV) > 0));REPLACE INTO t_DX44 (c_LGUf, c_Hlmf3w, c_ewZ, c_EwP, c_YBA7sBV) VALUES (-337373, 700752.2922865644, \'X6uz2_2\', \'jBG_3\', \'goHkluIlVWV39WFOlHFN_4\');INSERT OR ABORT INTO t_DX44 (c_LGUf, c_Hlmf3w, c_ewZ, c_EwP, c_YBA7sBV) VALUES (-332961, -615984.5192080662, \'NbEpbWe5OkP2G_2\', \'sbVgPrtyZ4FgrpMy0bq_3\', \'8yBj7AUGJm8qY7MpLB7_4\'), (63127, -375775.2781741079, \'AibeOCUJulJV92VB_102\', \'WEBgX0glMa4kn_103\', \'O2aZgLZbxNb3AUtP3_104\');INSERT OR FAIL INTO t_DX44 (c_LGUf, c_Hlmf3w, c_ewZ, c_EwP, c_YBA7sBV) VALUES (-423527, 351687.89635672374, \'P1CA_2\', \'EeQuTAomJEFY5Fl_3\', \'L_4\'), (1405102, -79679

In [81]:
analysis["query number"] = get_elements_by_pattern(results, '[ANALYSIS] QUERY PATH: "/workspaces/reducer/queries/query', '/original_test.sql"[END ANALYSIS]')

analysis["original"] = get_elements_by_pattern(results, '[ANALYSIS] ORIGINAL QUERY: "', '"[END ANALYSIS]')
analysis["delta debugged"] = get_elements_by_pattern(results, '[ANALYSIS] AFTER DELTA DEBUGGING: ', ' [END ANALYSIS]')
#analysis["reduced"] = []

analysis["#statements original"] = analysis["original"].apply(lambda x: x.count(";"))
analysis["#statements delta debugged"] = analysis["delta debugged"].apply(lambda x: x.count(";"))
analysis["#tokens original"] = analysis["original"].apply(lambda s: len(s.split(" ")))

analysis["DIFF orig-delta"] = analysis["#statements original"] - analysis["#statements delta debugged"]
analysis

,query number,original,delta debugged,#statements original,#statements delta debugged,#tokens original,DIFF orig-delta
0,2,CREATE TABLE IF NOT EXISTS t_DX44 (c_LGUf NUME...,CREATE TABLE IF NOT EXISTS t_DX44 (c_LGUf NUME...,18,5,249,13
